<a href="https://colab.research.google.com/github/smshozab/FieldMatrix/blob/main/Agriculture_Research_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install segmentation-models-pytorch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64

In [4]:
import cv2
import numpy as np
import torch
from torchvision import transforms
import segmentation_models_pytorch as smp

# Load the segmentation model (e.g., U-Net with a ResNet backbone)
model = smp.Unet(encoder_name='resnet34', encoder_weights='imagenet', in_channels=3, classes=1)
model.eval()  # Set model to evaluation mode

# Function to calculate GSD (Ground Sampling Distance)
def calculate_gsd(drone_height, sensor_width, image_width, focal_length):
    return (drone_height * sensor_width) / (focal_length * image_width)

# Function to calculate real-world distances
def calculate_distance(px_distance, gsd):
    return px_distance * gsd

# Load and preprocess the image
image_path = 'p5.jpeg'
image = cv2.imread(image_path)
image_resized = cv2.resize(image, (256, 256))  # Resize for the model

# Transform image for the model
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
image_input = transform(image_resized).unsqueeze(0)  # Add batch dimension

# Predict the segmentation mask
with torch.no_grad():
    mask = model(image_input)[0].squeeze().cpu().numpy()
mask = (mask > 0.5).astype(np.uint8)  # Threshold the mask

# Find contours of the segmented regions
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Calculate centroids
centroids = []
for contour in contours:
    M = cv2.moments(contour)
    if M['m00'] != 0:
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
        centroids.append((centroid_x, centroid_y))

# Parameters (replace with actual values)
drone_height = 200  # Height from which the image was taken (meters)
sensor_width = 12  # Width of the camera sensor (mm)
image_width = image.shape[1]  # Width of the image in pixels
focal_length = 8  # Focal length of the camera (mm)

# Calculate GSD
gsd = calculate_gsd(drone_height, sensor_width, image_width, focal_length)

# Calculate distances between plants
distances = []
for i, (cx1, cy1) in enumerate(centroids):
    for j, (cx2, cy2) in enumerate(centroids[i+1:]):
        px_distance = np.sqrt((cx2 - cx1)**2 + (cy2 - cy1)**2)
        real_distance = calculate_distance(px_distance, gsd)
        distances.append((i, i + j + 1, real_distance))

# Output the distances
for (i, j, distance) in distances:
    print(f"Distance between plant {i} and plant {j}: {distance:.2f} meters")
    # cv2.line(image, centroids[i], centroids[j], (0, 255, 0), 2)
    # mid_point = ((centroids[i][0] + centroids[j][0]) // 2, (centroids[i][1] + centroids[j][1]) // 2)
    # cv2.putText(image, f"{distance:.2f}m", mid_point, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

# # Save the segmented mask as an image
# segmented_image_path = 'segmented_image.png'
# cv2.imwrite(segmented_image_path, mask_resized * 255)  # Save as binary mask image


# Display the image with detections (optional)
for centroid in centroids:
    cv2.circle(image, (int(centroid[0]), int(centroid[1])), 5, (0, 255, 0), -1)
cv2.imshow('Segmented Plants', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Streaming output truncated to the last 5000 lines.
Distance between plant 145 and plant 196: 57.91 meters
Distance between plant 145 and plant 197: 51.54 meters
Distance between plant 145 and plant 198: 42.06 meters
Distance between plant 145 and plant 199: 34.56 meters
Distance between plant 145 and plant 200: 63.25 meters
Distance between plant 145 and plant 201: 60.57 meters
Distance between plant 145 and plant 202: 43.84 meters
Distance between plant 145 and plant 203: 34.25 meters
Distance between plant 145 and plant 204: 49.89 meters
Distance between plant 145 and plant 205: 59.66 meters
Distance between plant 145 and plant 206: 52.26 meters
Distance between plant 145 and plant 207: 42.02 meters
Distance between plant 145 and plant 208: 58.26 meters
Distance between plant 145 and plant 209: 61.80 meters
Distance between plant 145 and plant 210: 63.76 meters
Distance between plant 145 and plant 211: 22.51 meters
Distance between plant 145 and plant 212: 40.70 meters
Distance betwe

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow
